In [2]:
# from pathlib import Path
# import glob

# import numpy as np
# from matplotlib import pyplot as plt

# import os
# import sys

# from PyQt5.QtCore import *   # 5.9.2
# from PyQt5.QtGui import *
# from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QDialog
# from PyQt5.QtWidgets import *

# import cv2    # version required (at least): '3.4.4'


In [3]:
# %reset

In [2]:
from pathlib import Path
import glob

import numpy as np
from matplotlib import pyplot as plt

import os
import sys


from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QDialog, QButtonGroup
from PyQt5.QtWidgets import *

import cv2



In [2]:
# print(img_sources)      
# ! ls '{dir_per_img_src[img_sources[4]]}'

In [6]:
# ------------------------------------------------------------------------------------------------------------------

class ImageLabeler:
    
    def __init__(self):
        app = QApplication.instance()

        if app is None:
            app = QApplication(sys.argv)
        else:
            print('QApplication instance already exists: %s' % str(app))

        # glue the engine and GUI together    
        self.engine = LabelerEngine()
        self.gui = GUI(app, self.engine)        
        self.engine.gui = self.gui
        
        app.exec_()

# ------------------------------------------------------------------------------------------------------------------

class LabelerEngine:

    SELECT_IMG = 1
    DESELECT_IMG = -1
    
    class Config:
    
        workon_last_half = True
        
        # path to the dataset containing subdirectories with images
        #dataset_path = '/disks/data/datasets/selfie_project_datasets/selfie_dataset_versions/40k_copy'

        dir_base = '/disks/data/datasets/selfie_project_datasets/resized_ds/dataset_small/'

        class_names = ['selfie','wefie','mirror_selfie','non_selfie','non_mirror_selfie','non_wefie']
        
        selection_colors = {         
                        # BGR
            'selfie': (0, 0, 255),
            'mirror_selfie': (0, 128, 255),  #FF8000
            'wefie': (0, 255, 128),
            'non_selfie': (0, 255, 255),
            'non_mirror_selfie': (0, 255, 0), #00FF00
            'non_wefie': (255, 255, 0)
        
        }
        
        button_stylesheet = dict()
    
        button_stylesheet["ENABLE_STYLESHEET"] = \
                            """ QPushButton { border: 1px solid #007a94; border-radius: 6px; color:#ffffff; background-color: #007a94; min-width: 80px; height: 50px; }
                                QPushButton:pressed { background-color: qlineargradient(x1: 0, y1: 0, x2: 0, y2: 1, stop: 0 #008aa6, stop: 1 #008aa6);}
                                QPushButton:flat { border: none;}
                            """    

        button_stylesheet["DEFAULT_STYLESHEET"] = \
                             """ QPushButton { min-height: 50px;}
                             """
        
        button_stylesheet["DISABLE_STYLESHEET"] = \
                             """ QPushButton { border: 1px solid #808080; border-radius: 6px; color:#ffffff; background-color: #808080; min-width: 80px; }
                                 QPushButton:flat { border: none;}
                             """    
        
        button_stylesheet["selfie"] = \
                             """ QPushButton { border: 1px solid #FF0000; font-size:12pt; font-color: rgb(0,255,0); border-radius: 1px; color:#000000; background-color: #FF0000; min-width: 80px; height: 50px; }
                                QPushButton:pressed { background-color: qlineargradient(x1: 0, y1: 0, x2: 0, y2: 1, stop: 0 #008aa6, stop: 1 #008aa6);}
                                QPushButton:flat { border: none;}
                            """        
        
        button_stylesheet["selfie_ENABLED"] = \
                             """ QPushButton { border: 6px solid #000000; font-size:12pt; font-color: rgb(0,255,0); border-radius: 15px; color:#000000; background-color: #FF0000; min-width: 80px; height: 50px; }
                                QPushButton:pressed { background-color: qlineargradient(x1: 0, y1: 0, x2: 0, y2: 1, stop: 0 #008aa6, stop: 1 #008aa6);}
                                QPushButton:flat { border: none;}
                            """        
        
        button_stylesheet["mirror_selfie"] = \
                             """ QPushButton { border: 1px solid #FF0000; font-size:12pt; font-color: rgb(0,255,0); border-radius: 1px; color:#000000; background-color: #FF8000; min-width: 80px; height: 50px; }
                                QPushButton:pressed { background-color: qlineargradient(x1: 0, y1: 0, x2: 0, y2: 1, stop: 0 #008aa6, stop: 1 #008aa6);}
                                QPushButton:flat { border: none;}
                            """        
        
        button_stylesheet["mirror_selfie_ENABLED"] = \
                             """ QPushButton { border: 6px solid #000000; font-size:12pt; font-color: rgb(0,255,0); border-radius: 15px; color:#000000; background-color: #FF8000; min-width: 80px; height: 50px; }
                                QPushButton:pressed { background-color: qlineargradient(x1: 0, y1: 0, x2: 0, y2: 1, stop: 0 #008aa6, stop: 1 #008aa6);}
                                QPushButton:flat { border: none;}
                            """        
        

        button_stylesheet["wefie"] = \
                             """ QPushButton { border: 1px solid #80FF00; font-size:12pt; color: black; border-radius: 1px; color:#000000; background-color: #80FF00; min-width: 80px; height: 50px; }
                                QPushButton:pressed { background-color: qlineargradient(x1: 0, y1: 0, x2: 0, y2: 1, stop: 0 #008aa6, stop: 1 #008aa6);}
                                QPushButton:flat { border: none;}
                            """
        
        button_stylesheet["wefie_ENABLED"] = \
                             """ QPushButton { border: 6px solid #000000; font-size:12pt; color: black; border-radius: 15px; color:#000000; background-color: #80FF00; min-width: 80px; height: 50px; }
                                QPushButton:pressed { background-color: qlineargradient(x1: 0, y1: 0, x2: 0, y2: 1, stop: 0 #008aa6, stop: 1 #008aa6);}
                                QPushButton:flat { border: none;}
                            """

        button_stylesheet["non_selfie"] = \
                             """ QPushButton { border: 1px solid #FFFF00; font-size:12pt;  font-color: black; border-radius: 1px; color:#000000; background-color: #FFFF00; min-width: 80px; height: 50px; }
                                QPushButton:pressed { background-color: qlineargradient(x1: 0, y1: 0, x2: 0, y2: 1, stop: 0 #008aa6, stop: 1 #008aa6);}
                                QPushButton:flat { border: none;}
                            """

        button_stylesheet["non_selfie_ENABLED"] = \
                             """ QPushButton { border: 6px solid #000000; font-size:12pt;  font-color: black; border-radius: 15px; color:#000000; background-color: #FFFF00; min-width: 80px; height: 50px; }
                                QPushButton:pressed { background-color: qlineargradient(x1: 0, y1: 0, x2: 0, y2: 1, stop: 0 #008aa6, stop: 1 #008aa6);}
                                QPushButton:flat { border: none;}
                            """
        
        button_stylesheet["non_mirror_selfie"] = \
                             """ QPushButton { border: 1px solid #00FF00; font-size:12pt; font-color: rgb(0,255,0); border-radius: 1px; color:#000000; background-color: #00FF00; min-width: 80px; height: 50px; }
                                QPushButton:pressed { background-color: qlineargradient(x1: 0, y1: 0, x2: 0, y2: 1, stop: 0 #008aa6, stop: 1 #008aa6);}
                                QPushButton:flat { border: none;}
                            """        
        
        button_stylesheet["non_mirror_selfie_ENABLED"] = \
                             """ QPushButton { border: 6px solid #000000; font-size:12pt; font-color: rgb(0,255,0); border-radius: 15px; color:#000000; background-color: #00FF00; min-width: 80px; height: 50px; }
                                QPushButton:pressed { background-color: qlineargradient(x1: 0, y1: 0, x2: 0, y2: 1, stop: 0 #008aa6, stop: 1 #008aa6);}
                                QPushButton:flat { border: none;}
                            """      
        
        button_stylesheet["non_wefie"] = \
                             """ QPushButton { border: 1px solid #00FFFF; font-size:12pt;  font-color: black; border-radius: 1px; color:#000000; background-color: #00FFFF; min-width: 80px; height: 50px; }
                                QPushButton:pressed { background-color: qlineargradient(x1: 0, y1: 0, x2: 0, y2: 1, stop: 0 #008aa6, stop: 1 #008aa6);}
                                QPushButton:flat { border: none;}
                            """
 
        button_stylesheet["non_wefie_ENABLED"] = \
                             """ QPushButton { border: 6px solid #000000; font-size:12pt;  font-color: black; border-radius: 15px; color:#000000; background-color: #00FFFF; min-width: 80px; height: 50px; }
                                QPushButton:pressed { background-color: qlineargradient(x1: 0, y1: 0, x2: 0, y2: 1, stop: 0 #008aa6, stop: 1 #008aa6);}
                                QPushButton:flat { border: none;}
                            """
    
        img_src_to_dir = {
            'bing selfie' : f'{dir_base}bing/selfie',
            'google selfie' : f'{dir_base}google/selfie',
            'mshah selfie' : f'{dir_base}instagram/selfie',
            'flickr selfie' : f'{dir_base}flickr/selfie',
            
            'bing mirror selfie' : f'{dir_base}bing/mirror_selfie',
            'google mirror selfie' : f'{dir_base}google/mirror_selfie',
            
            'bing wefie' : f'{dir_base}bing/wefie',
            'google wefie' : f'{dir_base}google/wefie',
            
            'bing non_selfie faces' : f'{dir_base}bing/non_selfie/faces', 
            'bing non_selfie portr' : f'{dir_base}bing/non_selfie/portrait',
            'bing non_selfie m.shot' : f'{dir_base}bing/non_selfie/medium_shot',
            'bing non_selfie m.c.up' : f'{dir_base}bing/non_selfie/medium_close_up',
            'ggle non_selfie faces' : f'{dir_base}google/non_selfie/faces', 
            'ggle non_selfie portr' : f'{dir_base}google/non_selfie/portrait',
            'ggle non_selfie m.shot' : f'{dir_base}google/non_selfie/medium_shot',
            'ggle non_selfie m.c.up' : f'{dir_base}google/non_selfie/medium_close_up',
            'imgnet non_selfie h.sp' : f'{dir_base}imagenet/non_selfie/imagenet_homosapiens',
            'imgnet non_selfie h.rc' : f'{dir_base}imagenet/non_selfie/imagenet_humanrace',
            'instagram non_selfie' : f'{dir_base}instagram/non_selfie',
            'flickr non_selfie' : f'{dir_base}flickr/non_selfie',
            
            'bing non_mirror selfie' : f'{dir_base}bing/non_mirror_selfie',
            'google non_mirror selfie' : f'{dir_base}google/non_mirror_selfie',

            'bing non_wefie group ph' : f'{dir_base}bing/non_wefie/group_photo',
            'bing non_wefie group pic' : f'{dir_base}bing/non_wefie/group_picture',
            'bing non_wefie m.sh.gr.ph' : f'{dir_base}bing/non_wefie/medium_shot_group_photo',
            'bing non_wefie m.sh.gr.pic' : f'{dir_base}bing/non_wefie/medium_shot_group_picture',
            'bing non_wefie people' : f'{dir_base}bing/non_wefie/people',
            'ggle non_wefie group_ph' : f'{dir_base}google/non_wefie/group_photo',
            'ggle non_wefie group_pic' : f'{dir_base}google/non_wefie/group_picture',
            'ggle non_wefie m.sh.gr.ph' : f'{dir_base}google/non_wefie/medium_shot_group_photo',
            'ggle non_wefie m.sh.gr.pic' : f'{dir_base}google/non_wefie/medium_shot_group_picture',
            'ggle non_wefie people' : f'{dir_base}google/non_wefie/people'
        }

        img_src_to_class_label = {
            'bing selfie' : 'selfie',
            'google selfie' : 'selfie',
            'mshah selfie' : 'selfie',
            'flickr selfie' : 'selfie',
            
            'bing mirror selfie': 'mirror_selfie',
            'google mirror selfie': 'mirror_selfie',
            
            'bing wefie' : 'wefie',
            'google wefie' : 'wefie',
            
            'bing non_selfie faces' : 'non_selfie', 
            'bing non_selfie portr' : 'non_selfie',
            'bing non_selfie m.shot' : 'non_selfie',
            'bing non_selfie m.c.up' : 'non_selfie',
            'ggle non_selfie faces' : 'non_selfie', 
            'ggle non_selfie portr' : 'non_selfie',
            'ggle non_selfie m.shot' : 'non_selfie',
            'ggle non_selfie m.c.up' : 'non_selfie',
            'imgnet non_selfie h.sp' : 'non_selfie',
            'imgnet non_selfie h.rc' : 'non_selfie',
            'instagram non_selfie' : 'non_selfie',
            'flickr non_selfie' : 'non_selfie',
            
            'bing non_mirror selfie' : 'non_mirror_selfie',
            'google non_mirror selfie' : 'non_mirror_selfie',
            
            'bing non_wefie group ph' : 'non_wefie',
            'bing non_wefie group pic' : 'non_wefie',
            'bing non_wefie m.sh.gr.ph' : 'non_wefie',
            'bing non_wefie m.sh.gr.pic' : 'non_wefie',
            'bing non_wefie people' : 'non_wefie',
            'ggle non_wefie group_ph' : 'non_wefie',
            'ggle non_wefie group_pic' : 'non_wefie',
            'ggle non_wefie m.sh.gr.ph' : 'non_wefie',
            'ggle non_wefie m.sh.gr.pic' : 'non_wefie',
            'ggle non_wefie people' : 'non_wefie'
        }        
        

        
        img_sources = [str(key) for key in img_src_to_dir.keys()]            
        
        
        confirmed_dir = 'confirmed_'

        # image montage dimensions
        MONTAGE_COL_COUNT = 15
        MONTAGE_ROW_COUNT = 10

        # default size of individual images in the montage (height, width)
        IMSIZE = (150, 150)

        IMGRID_SHAPE = (MONTAGE_ROW_COUNT, MONTAGE_COL_COUNT)
        IMGRID_SIZE = np.prod(IMGRID_SHAPE)

        #class_names = [fname for dirpath,fname in [os.path.split(x) for x in glob.glob(os.path.join(dataset_path,'*'))] if not fname.startswith('confirmed_') ]
        #class_names = dir_selections
        
        #print([x for x in glob.glob(os.path.join(dataset_path,'*'))])
        
        #class_label = class_names[0]# if len(Config.class_names)>0 else None        
        
        class_label = None
        
        confirmed_img_dir_names = dict()
        confirmed_img_dirs = dict()
        
        @classmethod
        def get_class_label(cls, img_source):
            return cls.img_src_to_class_label[img_source]
        
        @classmethod
        def get_img_path(cls, img_source):
            return cls.img_src_to_dir[img_source]
        
        
        @classmethod
        def setup_source_paths(cls, img_source):

            #if not class_name is None:
            #    cls.class_label = class_name 
            #cls.dataset_path = Path(cls.dataset_path_str)
            #cls.img_path = os.path.join(cls.dataset_path, cls.class_label)
            cls.img_source = img_source
            cls.img_path = cls.get_img_path(img_source)
            
        @classmethod
        def setup_destination_paths(cls):
            
            # new directories under 'dataset_path' where filtered images will be moved to 
            #cls.moved_img_class_no_dirname = cls.class_label + '__no'
            cls.rejected_img_dir_name = 'rejections'
            cls.rejected_img_dir = os.path.join(cls.dir_base, os.path.join(cls.confirmed_dir, cls.rejected_img_dir_name) )

            # create directories for the moved images
            os.makedirs(cls.rejected_img_dir, exist_ok = True)
            
            for class_name in cls.class_names:
                cls.confirmed_img_dir_names[class_name] = class_name + '__confirmed'
                cls.confirmed_img_dirs[class_name] = os.path.join(cls.dir_base, os.path.join(cls.confirmed_dir, cls.confirmed_img_dir_names[class_name]) )
                os.makedirs(cls.confirmed_img_dirs[class_name], exist_ok = True)
            
            # full paths of directories where selected and unselected images will be moved to
            #cls.newdir_yes = os.path.join(cls.dataset_path, os.path.join(cls.confirmed_dir,cls.moved_img_class_yes) )
            #cls.newdir_no = os.path.join(cls.dataset_path, os.path.join(cls.confirmed_dir,cls.moved_img_class_no) )

                        
        @classmethod
        def set_class_label(cls, class_label):

            #if not class_name is None:
            #    cls.class_label = class_name 
            #cls.dataset_path = Path(cls.dataset_path_str)
            #cls.img_path = os.path.join(cls.dataset_path, cls.class_label)
            cls.class_label = class_label
            



    
    def __init__(self):
        
        self.config = LabelerEngine.Config
        self.reset_selections()
        self.config.setup_destination_paths()
        
        #self.config.setup_paths(class_name = None) # default class: the first one
        
        #self.load_image_names(self.config.class_label)
        
    
    def getClasses(self):        
        
        return self.config.class_names
    
    
    def getNextPanelImage(self, container_height, container_width):
        
        # process selected (and unselected) images
        #process_selected_imgs(self.selected_imgs_names, self.im_fnames_montage)
        
        self.current_montage_start_idx = self.current_montage_start_idx + 1

        montage_start_idx = self.montage_img_start_indices[self.current_montage_start_idx]

        # generate indices for the list of all images        
        img_count = len(self.img_files)

        # indices of the images in current montage    
        imgs_idx_montage = slice(montage_start_idx, min(montage_start_idx + self.config.IMGRID_SIZE, img_count) )
        
        # filenames of the images in the montage
        self.im_fnames_montage = self.img_files[imgs_idx_montage]

        im_height = container_height//self.config.IMGRID_SHAPE[0]
        im_width = container_width//self.config.IMGRID_SHAPE[1]
        self.config.IMSIZE = (im_height, im_width) 
        
        # build the montage image        
        self.image_montage = self._create_montage(self.im_fnames_montage, self.config.IMGRID_SHAPE, self.config.IMSIZE)
        
        return self.image_montage

    
    def getCurrentPanelImage(self):
        return self.image_montage
    
    def reset_selections(self):
        
        # lists containing the filenames of the selected images for each label
        self.selected_imgs_names = dict()
        self.selected_imgs_coords = dict()
        
        for class_name in self.config.class_names:
            self.selected_imgs_names[class_name] = []
            self.selected_imgs_coords[class_name] = []
            
        #self.selected_imgs_names = []
        #self.selected_imgs_coords = []

        
    def scantree(self, path, ext):
        """Recursively yield DirEntry objects for given directory."""
        for entry in os.scandir(path):
            if entry.is_dir(follow_symlinks=False):
                yield from self.scantree(entry.path, ext)  # see below for Python 2.x
            elif entry.path.endswith(ext):
                yield entry

    def get_img_file_names_in_dir(self, path, ext):
        return [entry.path for entry in self.scantree(path, ext)]
    
    def load_image_names(self, img_source):
        
        self.reset_selections()
        self.config.class_label = None
        self.gui.reset_label_buttons()
        
        self.config.setup_source_paths(img_source)
        
        # read in list of image files
        #self.img_files = glob.glob(os.path.join(self.config.img_path,'*.jpg'))
        #self.img_files = glob.glob(os.path.join(self.config.img_path,'**/*.jpg'), recursive=True)
        self.img_files = self.get_img_file_names_in_dir(os.path.join(self.config.img_path), '.jpg')

        if(self.config.workon_last_half):
            image_count = len(self.img_files)
            self.img_files = self.img_files[image_count//2:]
        
        # generate indices for the list of all images        
        im_indices = list(range(len(self.img_files)))
        #print(self.config.img_path)
        #print(len(im_indices))
        self.montage_img_start_indices = im_indices[::self.config.IMGRID_SIZE]
        self.current_montage_start_idx = -1
        
    
    def _create_montage(self, imglist, montage_shape, img_sz):
        '''
        creates montage from a list of image filename, montage shape, and per-image size.

        params:
            imglist (str): list of filepaths of the images
            montage_shape (tuple): contains the dimensions (row_count, col_count) of the montage
            img_sz (tuple): contains the dimensions (height, width [, depth) of each image in the montage

        returns:
            numpy ndarray containing the montage of images.
        '''
        
        # create an empty canvas for the montage
        canvas = np.zeros( (img_sz[0]*montage_shape[0], img_sz[1]*montage_shape[1], 3), np.uint8)

        # fill-up the montage row-by-row
        for icol in range(montage_shape[1]):
            for irow in range(montage_shape[0]):
                # x-y coords to linear index 
                img_idx = irow*montage_shape[1] + icol

                # read and resize the image; on failure use a red placeholder
                try:
                    img_fname = imglist[img_idx]
                    img = cv2.resize(cv2.imread(img_fname),(img_sz[1],img_sz[0]))
                except Exception as e:
                    img = np.zeros((*img_sz[:2],3), np.uint8)
                    img[::-1] = 255
                    print('file read error',img_fname,e)

                # populate the canvas with the image
                canvas[irow*img_sz[0]: (irow+1)*img_sz[0],
                      icol*img_sz[1]: (icol+1)*img_sz[1],:] = img

        # return the montage        
        return canvas
        
    def generate_moved_img_paths(self, selected_imgs_all_label, unselected_imgs):
        # generate paths of the moved (selected and unselected) images
        #newpaths_yes = [os.path.join(self.config.newdir_yes, fname) for fname in [ fname for path, fname in [os.path.split(filepath) for filepath in unselected_imgs ] ] ]
        #newpaths_no = [os.path.join(self.config.newdir_no, fname) for fname in [ fname for path, fname in [os.path.split(filepath) for filepath in selected_imgs ] ] ]
        
        newpaths_confirmed_imgs = []
        
        for class_name in self.config.class_names:
            for filepath in self.selected_imgs_names[class_name]:
                #print(filepath)
                #print(os.path.split(filepath))
                #print(self.selected_imgs_names[class_name])
                #set_trace()
                path, fname = os.path.split(filepath)
                newpaths_confirmed_imgs.append( os.path.join(self.config.confirmed_img_dirs[class_name], fname) )                    

        newpaths_rejected = [os.path.join(self.config.rejected_img_dir, fname) for fname in [ fname for path, fname in [os.path.split(filepath) for filepath in unselected_imgs ] ] ]

        return (newpaths_confirmed_imgs, newpaths_rejected)
                                    
    def process_selected_imgs(self):
        '''
        Move selected and unselected images to other, separate directories

        params:
            selected_imgs (list): list of selected images' paths
            images (list): list of all images' paths from the currently shown montage

        returns:
            Nothing
        '''

        selected_imgs_all_label = []
        for class_name in self.config.class_names:
            selected_imgs_all_label.extend(self.selected_imgs_names[class_name])
            
        # convert list to sets to eliminate duplicates
        #selected_imgs = set(self.selected_imgs_names)
        # find the list of unselected images
        unselected_imgs = list(set(self.im_fnames_montage)-set(selected_imgs_all_label))

        newpaths_confirmed_imgs, newpaths_rejected = self.generate_moved_img_paths(selected_imgs_all_label, unselected_imgs)
        
             
#         for class_name in self.config.class_names:
#             print(class_name)
#             for i, c in enumerate(self.selected_imgs_names[class_name]):
#                 print('\t',newpaths_confirmed_imgs[i])
#             print()
            
        # move the selected and unselected images
        [os.rename( fr_file, to_file) for fr_file, to_file in zip(selected_imgs_all_label, newpaths_confirmed_imgs)]
        [os.rename( fr_file, to_file) for fr_file, to_file in zip(unselected_imgs, newpaths_rejected)]

        # list containing the filenames of the selected images
        self.reset_selections()
 
    
    def act_on_image_at(self, click_x, click_y, action):
        
        # get montage image-space coordinates of the selected image
        def get_rectangle_corners(top_left, im_size, line_width):
            return ( (top_left[0]*im_size[1] + line_width//2, 
                      top_left[1]*im_size[0] + line_width//2),
                     ((top_left[0]+1)*im_size[1] - line_width//2, 
                      (top_left[1]+1)*im_size[0]- line_width//2) 
                   )
    
        def draw_border(im_coords, line_width, color):
            # montage image-space coordinates
            corners = get_rectangle_corners(im_coords, self.config.IMSIZE, line_width)

            # index of the selected image in the montage
            im_idx = im_coords[1]*self.config.IMGRID_SHAPE[1] + im_coords[0]

            # the file name of the selected image
            im_name = self.im_fnames_montage[im_idx]
            #print(im_idx, im_name)

            # draw a rectangular border around the selected image
            cv2.rectangle(self.image_montage, corners[0], corners[1], color, line_width )            

            return im_name


        if(not self.gui.is_img_loaded or self.config.class_label is None):
            return
        
        
        # width of the rectangle border to be drawn around the selected image
        line_width = 6
            
        if(action == self.SELECT_IMG):
            # grid-space (zero-indexed) coordinates of the selected image
            im_coords = (click_x//self.config.IMSIZE[1], click_y//self.config.IMSIZE[0])
            
            im_name = draw_border(im_coords, line_width, self.config.selection_colors[self.config.class_label])

            self.selected_imgs_coords[self.config.class_label].append(im_coords)

            self.selected_imgs_names[self.config.class_label].append(im_name)
                
        elif(action == self.DESELECT_IMG and len(self.selected_imgs_names[self.config.class_label]) > 0):

            # remove the image file name from no-move queue
            im_name = self.selected_imgs_names[self.config.class_label].pop()

            # retrieve the coordinate of the previous selection
            latest_img_coords = self.selected_imgs_coords[self.config.class_label].pop()

            # draw a green border around the de-selected image
            draw_border(latest_img_coords, line_width, (0,0,0))   
        
# ------------------------------------------------------------------------------------------------------------------        
class MontageLabel(QLabel):
    
    def __init__(self, parent = None):
        super(MontageLabel, self).__init__(parent)
        self.parent = parent
        
    def mouseReleaseEvent(self, ev: QMouseEvent):

        #print(self.parent.engine.config.class_label)
        
        if(not self.parent.is_img_loaded):
            return
                
        if(self.parent.engine.config.class_label is None):
            return 
        
        pos = ev.pos()

        self._ActiveButton = ev.button()

        if self._ActiveButton == Qt.LeftButton:
            self.parent.engine.act_on_image_at(pos.x(), pos.y(), self.parent.engine.SELECT_IMG)
            self.parent.redraw_img()
        elif self._ActiveButton == Qt.RightButton:           
            self.parent.engine.act_on_image_at(pos.x(), pos.y(), self.parent.engine.DESELECT_IMG)
            self.parent.redraw_img()
            
# ------------------------------------------------------------------------------------------------------------------   

class GUI(QWidget):
       
    def __init__(self, app, engine, parent=None):
        
        super(GUI, self).__init__(parent)

        self.engine = engine
        self.app = app
        
        screen = app.primaryScreen()
        size = screen.size()
        self.size = (size.width(), size.height())
        rect = screen.availableGeometry()
        self.visible_sz = (rect.width(), rect.height())

        # container for the montage image
        layout = QGridLayout(self)
        self.label = MontageLabel(self)#QLabel(self)
        self.label.setStyleSheet("background-color: white; inset grey; min-height: 200px;")
        self.label.setFrameShape(QFrame.Panel)
        self.label.setFrameShadow(QFrame.Sunken)
        
        self.summary = QLabel(self)
        self.confirm_labels_button = QPushButton("Confirm labels", self) 
        self.confirm_labels_button.clicked.connect(self.process_currentset_and_load_nextset)
        self.confirm_labels_button.setEnabled(False)
        
        self.buttons_label = QLabel(self)
        self.buttons_label.setText("Select label to Apply")
        self.img_src_label = QLabel(self)
        self.img_src_label.setText("Select image source to use")
        
        class_names = engine.getClasses()
                
        vlayout = QVBoxLayout()
        
        self.comboBox = QComboBox(self)
        self.comboBox.addItem("")
        
        for class_name in self.engine.config.img_sources:
            self.comboBox.addItem(class_name)

        # make the first item non-selectable    
        model = self.comboBox.model()
        first_index = model.index(0, self.comboBox.modelColumn(), self.comboBox.rootModelIndex());
        model.itemFromIndex(first_index).setSelectable(False)   
        
        self.comboBox.activated[str].connect(self.class_combobox_selected)
        
        
        vlayout.addWidget(self.img_src_label)        
        vlayout.addWidget(self.comboBox)
        vlayout.addStretch(1)

        vlayout.addWidget(self.buttons_label)
        vlayout.addStretch(0.5)
        
        self.setup_label_buttons()        
        for button in self.class_buttons:
            vlayout.addWidget(button)
        vlayout.addStretch(1)        
        vlayout.addWidget(self.confirm_labels_button)
        
        vlayout.addStretch(2)
        vlayout.addWidget(self.summary)
        vlayout.addStretch(2)
        
        
        layout.addWidget(self.label,0,0)
        layout.addItem(vlayout,0,1)
        
        layout.setColumnStretch(0, 15)
        layout.setColumnStretch(1, 1)

        self.setLayout(layout)
                
        self.showMaximized()
        
        self.is_img_loaded = False
 
    def set_all_button_enable_state(self, state):        
        for button in self.class_buttons:
            button.setEnabled(state)    
        self.confirm_labels_button.setEnabled(state)

    def reset_label_buttons(self):
        for button in self.class_buttons:
                button.setStyleSheet(self.engine.config.button_stylesheet[button.text()])
        self.summary.setText("")
        
    #@pyqtSlot()
    def label_button_click_handler(self, clicked_button):
        #print(clicked_button.text())
        class_label = clicked_button.text()
        #self.engine.label = clicked_button.text()
        for button in self.class_buttons:
            if not button is clicked_button:
                button.setStyleSheet(self.engine.config.button_stylesheet[button.text()])
        clicked_button.setStyleSheet(self.engine.config.button_stylesheet[clicked_button.text()+"_ENABLED"])
        
        self.engine.config.set_class_label(class_label)
        self.refreshSummary()
        
    def setup_label_buttons(self):
        self.class_buttons = [QPushButton(class_label, self) for class_label in self.engine.getClasses()] 
        self.class_button_group = QButtonGroup(self)
        self.class_button_group.setExclusive(True)
        for button in self.class_buttons:
            button.setStyleSheet(self.engine.config.button_stylesheet[button.text()])
            self.class_button_group.addButton(button)      
            
        self.class_button_group.buttonClicked.connect(self.label_button_click_handler)
        
    def _np2pixmap(self, npimg):
        pilimg = cv2.cvtColor(npimg, cv2.COLOR_BGR2RGB)

        height, width, byteValue = npimg.shape
        byteValue = byteValue * width
        mQImage = QImage(pilimg, width, height, byteValue, QImage.Format_RGB888)

        pixmap = QPixmap(mQImage)
                                               
        return pixmap                                               
                                               
    def class_combobox_selected(self, text):        
        self.set_all_button_enable_state(False)        
        self.engine.load_image_names((str(self.comboBox.currentText())))
        np_img = self.engine.getNextPanelImage(self.label.height(), self.label.width())        
        
        self.label.setPixmap(self._np2pixmap(np_img))
        self.label.setFocus()
        #self.refreshSummary()
        self.repaint()
        self.is_img_loaded = True
        self.set_all_button_enable_state(True)
        
    def redraw_img(self):        
        self.set_all_button_enable_state(False)                
        np_img = self.engine.getCurrentPanelImage()        
        
        self.label.setPixmap(self._np2pixmap(np_img))
        self.label.setFocus()
        self.repaint()
        self.set_all_button_enable_state(True)        

    def refreshSummary(self):
        if(not self.engine.config.class_label is None):
            txt = '#' + self.engine.config.class_label + f': {sum([len(files) for r, d, files in os.walk(self.engine.config.confirmed_img_dirs[self.engine.config.class_label])])}<br>' \
            +'#rejections ' + f': {sum([len(files) for r, d, files in os.walk(self.engine.config.rejected_img_dir)])}'

            self.summary.setText(txt)
        

    def process_currentset_and_load_nextset(self):
        
        if(not self.is_img_loaded ):# or self.engine.config.class_label is None):
            return
        
        self.set_all_button_enable_state(False)                
        self.engine.process_selected_imgs()
        
        np_img = self.engine.getNextPanelImage(self.label.height(), self.label.width())        

        self.label.setPixmap(self._np2pixmap(np_img))
        
        self.label.setFocus()
        self.refreshSummary()
        self.repaint()
        self.set_all_button_enable_state(True)        

        
    def keyPressEvent(self, QKeyEvent):
        
        if(self.is_img_loaded and 'n' == QKeyEvent.text()):
            super(GUI, self).keyPressEvent(QKeyEvent)

            self.process_currentset_and_load_nextset()
            
# ------------------------------------------------------------------------------------------------------------------

if __name__=="__main__":
    ImageLabeler()

In [72]:
# Config.dataset_path_str

In [73]:
# flist = ['/disks/data/datasets/selfie_project_datasets/tmp_copy/bing/selfie/selfie_images_pastmonth/bing_selfie_pastmonth_000337.jpg', '/disks/data/datasets/selfie_project_datasets/tmp_copy/bing/selfie/selfie_images_pastmonth/bing_selfie_pastmonth_000273.jpg', '/disks/data/datasets/selfie_project_datasets/tmp_copy/bing/selfie/selfie_images_pastmonth/bing_selfie_pastmonth_000438.jpg']

# for f in flist:
#     a, b= os.path.split(f)


In [74]:
# img = cv2.imread('/disks/data/datasets/selfie_project/selfie_dataset_versions/100k_copy/non_selfie/balkenende.jpg')

In [75]:
# Config.setup_config()

In [76]:
# a

In [77]:
# class Config:
    
#         # path to the dataset containing subdirectories with images
#         dataset_path_str = '/disks/data/datasets/selfie_project/selfie_dataset_versions/100k_copy'

#         # sub directory where the to-be inspected reside
#         #class_label = 'selfie'

#         # flag deciding whether to show the selected image in a separate window
#         show_selected_images = False

#         # image montage parameters
#         MONTAGE_COL_COUNT = 5
#         MONTAGE_ROW_COUNT = 5

#         # size of individual images in the montage (width, height)
#         IMSIZE = (150,150)

#         IMGRID_SHAPE = (MONTAGE_COL_COUNT, MONTAGE_ROW_COUNT)
#         IMGRID_SIZE = np.prod(IMGRID_SHAPE)

#         class_names = [fname for dirpath,fname in [os.path.split(x) for x in glob.glob(os.path.join(dataset_path_str,'*'))] if not fname.startswith('yes_') and not fname.startswith('no_') ]

#         class_label = class_names[0]# if len(Config.class_names)>0 else None        

#         @staticmethod
#         def setup_paths(class_name = None):

#             if not class_name is None:
#                 Config.class_label = class_name 

#             # new directories under 'dataset_path' where filtered images will be moved to 
#             Config.moved_img_class_no = 'no_' + Config.class_label
#             Config.moved_img_class_yes = 'yes_' + Config.class_label

#             Config.dataset_path = Path(Config.dataset_path_str)
#             Config.img_path = Config.dataset_path/Config.class_label

#             # full paths of directories where selected and unselected images will be moved to
#             Config.newdir_yes = Config.dataset_path/Config.moved_img_class_yes
#             Config.newdir_no = Config.dataset_path/Config.moved_img_class_no

#             # create directories for the moved images
#             os.makedirs(Config.newdir_yes, exist_ok = True)
#             os.makedirs(Config.newdir_no, exist_ok = True)

In [78]:
# Config.setup_paths()

In [79]:
# Config.moved_img_class_no

In [80]:
# LabelerEngine.Config.setup_paths()

